In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from sklearn.metrics import accuracy_score

# Prepair Data

In [4]:
df = pd.read_csv('data/train.csv')

df_X = df.drop('label', axis=1) 
df_y = df['label']

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=20) 

X = np.array(X_train / 255.0) # нормализуем данные, т.к. иначе ф-я для x>2 уже близка к 1
y = np.array([y_train]).T

X_val = np.array(X_test / 255.0) # нормализуем данные, т.к. иначе ф-я для x>2 уже близка к 1
y_val = np.array([y_test]).T

# for MSE
y_onehotenc = OneHotEncoder().fit_transform(y).toarray()
y_val_onehotenc = OneHotEncoder().fit_transform(y_val).toarray()

# Simple Keras model

In [22]:
model = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=keras.activations.sigmoid),
    keras.layers.Dense(10, activation=keras.activations.sigmoid)
])

model.compile(optimizer=keras.optimizers.SGD(lr=0.1, momentum=0.9), 
              loss='MSE',
              metrics=['accuracy'])

In [25]:
model.fit(X, y_onehotenc, epochs=200, batch_size=32, validation_data=(X_val, y_val_onehotenc))
# Epoch 200/200
# 33600/33600 [==============================] - 5s 157us/step - loss: 8.4901e-04 - acc: 0.9951 - val_loss: 0.0045 - val_acc: 0.9746

Train on 33600 samples, validate on 8400 samples
Epoch 1/200
33600/33600 [==============================] - 4s 117us/step - loss: 0.0018 - acc: 0.9914 - val_loss: 0.0048 - val_acc: 0.9729
Epoch 2/200
33600/33600 [==============================] - 4s 107us/step - loss: 0.0018 - acc: 0.9915 - val_loss: 0.0048 - val_acc: 0.9729
Epoch 3/200
33600/33600 [==============================] - 4s 108us/step - loss: 0.0018 - acc: 0.9915 - val_loss: 0.0048 - val_acc: 0.9727
Epoch 4/200
33600/33600 [==============================] - 4s 109us/step - loss: 0.0018 - acc: 0.9915 - val_loss: 0.0048 - val_acc: 0.9732
Epoch 5/200
33600/33600 [==============================] - 3s 103us/step - loss: 0.0018 - acc: 0.9916 - val_loss: 0.0048 - val_acc: 0.9731
Epoch 6/200
33600/33600 [==============================] - 3s 101us/step - loss: 0.0018 - acc: 0.9916 - val_loss: 0.0048 - val_acc: 0.9732
Epoch 7/200
33600/33600 [==============================] - 3s 101us/step - loss: 0.0018 - acc: 0.9916 - val_loss: 0.0

In [24]:
df_test = pd.read_csv('data/test.csv')

test = np.array(df_test) / 255.0

predict_matrix = model.predict(test)
predict_vector = predict_matrix.argmax(axis=1).T

predicted_df = pd.DataFrame(predict_vector,
                            index = np.arange(1, predict_vector.shape[0] + 1),
                            columns=['Label'])
predicted_df.to_csv('submission.csv', index_label='ImageId')
    

# Keras: adam, antropy

In [10]:
model_1 = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(392, activation=keras.activations.relu),
    keras.layers.Dense(256, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(32, activation=keras.activations.sigmoid),
    keras.layers.Dense(10, activation=keras.activations.softmax) ])

model_1.compile(optimizer=keras.optimizers.Adamax(lr=0.01, beta_1=0.95, beta_2=0.999, epsilon=None, decay=0.0), 
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [9]:
# model_1 = keras.models.Sequential([
#     keras.layers.Flatten(),
#     keras.layers.Dense(392, activation=keras.activations.relu),
#     keras.layers.Dense(256, activation=keras.activations.relu),
#     keras.layers.Dense(64, activation=keras.activations.relu),
#     keras.layers.Dense(32, activation=keras.activations.sigmoid),
#     keras.layers.Dense(10, activation=keras.activations.softmax) ])

# model_1.compile(optimizer=keras.optimizers.Adamax(lr=0.01, beta_1=0.95, beta_2=0.999, epsilon=None, decay=0.0), 
#               loss=keras.losses.categorical_crossentropy,
#               metrics=['accuracy'])

# kaggle - 0.98028

model_1.fit(X, y_onehotenc, epochs=10, batch_size=32, validation_data=(X_val, y_val_onehotenc))



Train on 33600 samples, validate on 8400 samples
Epoch 1/10
33600/33600 [==============================] - 8s 247us/step - loss: 0.0243 - acc: 0.9952 - val_loss: 0.1361 - val_acc: 0.9749
Epoch 2/10
33600/33600 [==============================] - 8s 243us/step - loss: 0.0267 - acc: 0.9946 - val_loss: 0.1471 - val_acc: 0.9739
Epoch 3/10
33600/33600 [==============================] - 8s 241us/step - loss: 0.0204 - acc: 0.9960 - val_loss: 0.1446 - val_acc: 0.9732
Epoch 4/10
33600/33600 [==============================] - 8s 242us/step - loss: 0.0226 - acc: 0.9957 - val_loss: 0.1389 - val_acc: 0.9739
Epoch 5/10
33600/33600 [==============================] - 9s 255us/step - loss: 0.0255 - acc: 0.9951 - val_loss: 0.1622 - val_acc: 0.9704
Epoch 6/10
33600/33600 [==============================] - 9s 262us/step - loss: 0.0321 - acc: 0.9942 - val_loss: 0.1565 - val_acc: 0.9713
Epoch 7/10
33600/33600 [==============================] - 9s 265us/step - loss: 0.0231 - acc: 0.9958 - val_loss: 0.1517 - v

In [61]:
df_test = pd.read_csv('data/test.csv')

test = np.array(df_test) / 255.0

predict_matrix = model_1.predict(test)
predict_vector = predict_matrix.argmax(axis=1).T

predicted_df = pd.DataFrame(predict_vector,
                            index = np.arange(1, predict_vector.shape[0] + 1),
                            columns=['Label'])
predicted_df.to_csv('submission.csv', index_label='ImageId')

# CNN